In [9]:
import sqlite3
import pandas as pd
import re
import json

conn = sqlite3.connect("data.sqlite3")

df_climbs = pd.read_sql_query("SELECT uuid, frames, angle FROM climbs", conn)
df_climb_stats = pd.read_sql_query("SELECT climb_uuid, display_difficulty FROM climb_stats", conn)

conn.close()

# Merge tables
df = pd.merge(df_climbs, df_climb_stats, left_on="uuid", right_on="climb_uuid", how="inner")
df.drop(columns=["climb_uuid"], inplace=True)

# Parse frames
def parse_frames(frames):
    pattern = r"p(\d{4})r(\d{2})"
    matches = re.findall(pattern, frames)
    color_mapping = {"12": "green", "13": "blue", "14": "purple", "15": "yellow"}
    
    holds = [{"hold_id": int(hold_id), "hold_color": color_mapping.get(color, "unknown")}
             for hold_id, color in matches]
    
    return json.dumps(holds)
    
df["holds"] = df["frames"].apply(parse_frames)

# CSV
df.to_csv("climbing_problems_with_holds.csv", index=False)

print(df.head())

                               uuid  \
0  002047402B6941CEA5ED7BB09FBFE14D   
1  002ED50792A94E5EB2127D59E167B2EE   
2  004FB7A0C0754DA98634C5EE4D985D9A   
3  004FB7A0C0754DA98634C5EE4D985D9A   
4  00683d10a8e246b3a106531c8573f13c   

                                              frames  angle  \
0  p1145r12p1146r12p1149r13p1186r13p1201r13p1256r...    NaN   
1  p1123r12p1139r13p1155r12p1171r13p1187r13p1203r...    NaN   
2  p1081r15p1090r13p1091r13p1094r12p1097r13p1098r...    NaN   
3  p1081r15p1090r13p1091r13p1094r12p1097r13p1098r...    NaN   
4  p1141r15p1142r12p1144r15p1179r12p1192r13p1246r...    NaN   

   display_difficulty                                              holds  
0                18.0  [{"hold_id": 1145, "hold_color": "green"}, {"h...  
1                25.0  [{"hold_id": 1123, "hold_color": "green"}, {"h...  
2                29.0  [{"hold_id": 1081, "hold_color": "yellow"}, {"...  
3                22.0  [{"hold_id": 1081, "hold_color": "yellow"}, {"...  
4          